In [1]:
import os
import torch
from torchvision import transforms # to transform the data
from torchvision import datasets # to load the data
from torch.utils.data import DataLoader # to work with the data
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, f1_score, recall_score, precision_score
from torchvision import models # to get ResNet model
from utils import evaluate_model

<h1 style="color: orange;">Paths</h1>

In [2]:
current_path = os.getcwd()
current_path = os.path.dirname(current_path)
data_dir = os.path.join(current_path, "3-Train-Val-Test-Split")
ml_dir = os.path.join(current_path, "4-Machine-Learning")
model_dir = os.path.join(ml_dir, "Base-Model")
metrics_dir = os.path.join(ml_dir, "metrics")

<h2 style="color: orange;">Processing Testing Images</h2>

In [3]:
input_size = 224
batch_size = 64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_transforms = {
    'test': transforms.Compose([
        transforms.Resize(int(input_size * 1.1)),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

<h2 style="color: orange;">Loading</h2>

In [4]:
image_datasets = {
    'test': datasets.ImageFolder(
        os.path.join(data_dir, 'test'), transform=data_transforms['test']
    )
}

<h2 style="color: orange;">Loader</h2>

In [5]:
dataloaders = {
    'test': DataLoader(image_datasets['test'], batch_size=batch_size, shuffle=False, num_workers=4)
}

In [6]:
class_names = image_datasets['test'].classes
print("Classes:", class_names)
print("Test dataset size:", len(image_datasets['test']))


Classes: ['asteroid', 'comet', 'galaxy', 'nebula', 'planet', 'star']
Test dataset size: 600


<h2 style="color: orange;">ResNet50 Model</h2>

In [7]:
base_model_path = os.path.join(model_dir, "best_model.pth")
resnet = models.resnet50(weights=None)
num_ftrs = resnet.fc.in_features
resnet.fc = torch.nn.Linear(num_ftrs, len(class_names))
resnet.load_state_dict(torch.load(base_model_path))
resnet = resnet.to(device)
print("Base model loaded.")

Base model loaded.


/var/folders/7z/dp25t9yj1t7fql93jpgg11jm0000gn/T/ipykernel_48941/2345314154.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  resnet.load_state_dict(torch.load(base_model_

<h2 style="color: orange;">Evaluate</h2>

In [8]:
# Evaluate the baseline model
evaluate_model(
    model=resnet,
    dataloaders=dataloaders,
    device=device,
    class_names=class_names,
    model_name="Baseline ResNet50",
    architecture_details="Learning Rate: 1e-3, Epochs: 20\nBatch Size: 64, Optimizer: Adam"
)

Metrics saved to: /Users/isi.mube/Ironhack/Data-Curriculum/DSML-Master/Cosmic-Morphology/4-Machine-Learning


{'accuracy': 0.445,
 'f1_score': 0.44264996022496206,
 'recall': 0.445,
 'precision': 0.44245466122572197,
 'confusion_matrix_path': '/Users/isi.mube/Ironhack/Data-Curriculum/DSML-Master/Cosmic-Morphology/4-Machine-Learning/Baseline ResNet50_confusion_matrix.png',
 'classification_report_path': '/Users/isi.mube/Ironhack/Data-Curriculum/DSML-Master/Cosmic-Morphology/4-Machine-Learning/Baseline ResNet50_classification_report.png'}

<h2 style="color: orange;">Fine-Tuned Model</h2>

In [9]:
fine_dir = os.path.join(ml_dir, "Fine-Tuned")
fine_tuned_model_path = os.path.join(fine_dir, "fine_tuned_model.pth")
fine_tuned_model = models.resnet50(weights=None)
fine_tuned_model.fc = torch.nn.Linear(num_ftrs, len(class_names))
fine_tuned_model.load_state_dict(torch.load(fine_tuned_model_path))
fine_tuned_model = fine_tuned_model.to(device)
print("Fine-tuned model loaded.")

Fine-tuned model loaded.


/var/folders/7z/dp25t9yj1t7fql93jpgg11jm0000gn/T/ipykernel_48941/1601869459.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  fine_tuned_model.load_state_dict(torch.load(f

<h2 style="color: orange;">Evaluate</h2>

In [10]:
# Evaluate the fine-tuned model
evaluate_model(
    model=fine_tuned_model,
    dataloaders=dataloaders,
    device=device,
    class_names=class_names,
    model_name="Fine-Tuned ResNet50",
    architecture_details="Learning Rate: 1e-4, Epochs: 10\nBatch Size: 64, Optimizer: Adam (fine-tuned)"
)

Metrics saved to: /Users/isi.mube/Ironhack/Data-Curriculum/DSML-Master/Cosmic-Morphology/4-Machine-Learning


{'accuracy': 0.4483333333333333,
 'f1_score': 0.44387468949067727,
 'recall': 0.4483333333333333,
 'precision': 0.446137965373033,
 'confusion_matrix_path': '/Users/isi.mube/Ironhack/Data-Curriculum/DSML-Master/Cosmic-Morphology/4-Machine-Learning/Fine-Tuned ResNet50_confusion_matrix.png',
 'classification_report_path': '/Users/isi.mube/Ironhack/Data-Curriculum/DSML-Master/Cosmic-Morphology/4-Machine-Learning/Fine-Tuned ResNet50_classification_report.png'}